## Observations and Insights 

In [105]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Enables chart interactivity in JN
%matplotlib inline

In [94]:
# Combine the data into a single dataset
# Notes: pd.merge merges two data files and uses "Mouse ID" fields as unique ID to tie data sets together.
# Notes: pandas dataframe = "all_data_df" so pull data and do analysis from this Pandas data set.
# Notes: Outer join = return all rows from both tables, join from left table using keys that match to the right.

## DATAFRAME
all_data_df = pd.merge(mouse_metadata, study_results, how="outer", on=["Mouse ID", "Mouse ID"])
#all_data_df.head()
print(all_data_df)

# Display the data table for preview

     Mouse ID Drug Regimen   Sex  Age_months  Weight (g)  Timepoint  \
0        k403     Ramicane  Male          21          16          0   
1        k403     Ramicane  Male          21          16          5   
2        k403     Ramicane  Male          21          16         10   
3        k403     Ramicane  Male          21          16         15   
4        k403     Ramicane  Male          21          16         20   
...       ...          ...   ...         ...         ...        ...   
1888     z969     Naftisol  Male           9          30         25   
1889     z969     Naftisol  Male           9          30         30   
1890     z969     Naftisol  Male           9          30         35   
1891     z969     Naftisol  Male           9          30         40   
1892     z969     Naftisol  Male           9          30         45   

      Tumor Volume (mm3)  Metastatic Sites  
0              45.000000                 0  
1              38.825898                 0  
2           

### Cleaning the data for duplicates

In [73]:
# Checking the number of mice based on Mouse ID
# Note: # total mice = 249

## VARIABLE
total_mice = len(all_data_df["Mouse ID"].unique())
print (total_mice)

249


In [74]:
# Count the number of times the value of Mouse ID is listed using .value.counts (vs just .counts)
# Most should have a count of 1-10 timepoints. Anything higher indicates duplicate timepoint data
all_data_df["Mouse ID"].value_counts()

g989    13
x581    10
i901    10
o331    10
a251    10
        ..
b447     1
d133     1
o848     1
l872     1
t573     1
Name: Mouse ID, Length: 249, dtype: int64

In [98]:
# OPTION A (see option B at end for alternate code)
# Get all the data for the duplicate mouse ID. 
# Note: This looks for g989 in Mouse ID and assigns it to "badmice".
bad_mice_data_df = all_data_df.set_index("Mouse ID")
badmice_df = bad_mice_data_df.loc["g989", :]
badmice_df.head()

,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
Mouse ID,,,,,,,
g989,Propriva,Female,21,26,0,45.000000,0
g989,Propriva,Female,21,26,0,45.000000,0
g989,Propriva,Female,21,26,5,48.786801,0
g989,Propriva,Female,21,26,5,47.570392,0
g989,Propriva,Female,21,26,10,51.745156,0


In [99]:
# Remove bad mouse data from entire (all_data_df) and create a new clean dataframe (all_data_clean_df).
# Note: # mice = 248 - and mouse G989 is still in the bucket but the duplicate data is gone.
# POTATO - unsure if I need to reset the index below. g989 is part of the index so may need to reset it.

clean_data_mice_df = bad_mice_data_df.drop(index = "g989")
all_data_clean_df = clean_data_mice_df.reset_index()
all_data_clean_df.head()
#print (all_data_clean_df)

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [101]:
# Counts number of "clean mice" in the new dataframe (all_data_clean_df)
# Note: # total mice = 249 | clean mice = 248
clean_mice_df = len(all_data_clean_df["Mouse ID"].unique())
print (clean_mice)

248


## Summary Statistics

In [103]:
# Generate a summary statistics table of mean, median, variance, standard deviation, 
# and SEM (standard error) of the tumor volume for each regimen

summarydrug = all_data_clean_df.groupby("Drug Regimen")
#summarydrug.head()

# Variables for data
tumor_mean = summarydrug ["Tumor Volume (mm3)"].mean ()
tumor_median = summarydrug ["Tumor Volume (mm3)"].median ()
tumor_variance = summarydrug ["Tumor Volume (mm3)"].var ()
tumor_std = summarydrug ["Tumor Volume (mm3)"].std ()
tumor_sem = summarydrug ["Tumor Volume (mm3)"].sem ()


# Summarization
summary_tumor_vol = pd.DataFrame({"Mean": tumor_mean, 
                                  "Median": tumor_median,
                                  "Variance": tumor_variance,
                                  "Standard Deviation": tumor_std, 
                                  "SEM (Std Error)": tumor_sem})

summary_tumor_vol.head()

#POTATO - START HERE. Is the calculatin right?If so, format the table. Also remember you renamed dataframes to _df
#.pd dataframe isn't right above - rename?

,Mean,Median,Variance,Standard Deviation,SEM (Std Error)
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,54.331565,52.509285,66.173479,8.134708,0.596466


In [79]:
# Use groupby and summary statistical methods to calculate the 
# following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.

In [80]:
# Generate a summary statistics table of mean, median, variance, standard deviation, 
# and SEM of the tumor volume for each regiment

# Using the aggregation method, produce the same summary statistics in a single line


In [104]:
%who_ls

#Lists

['all_data',
 'all_data_clean',
 'all_data_clean_df',
 'all_data_df',
 'bad_mice_data',
 'bad_mice_data_df',
 'badmice',
 'badmice_df',
 'clean_data_mice',
 'clean_data_mice_df',
 'clean_mice',
 'clean_mice_df',
 'mouse_metadata',
 'mouse_metadata_path',
 'np',
 'pd',
 'plt',
 'st',
 'study_results',
 'study_results_path',
 'summary_df',
 'summary_tumor_vol',
 'summarydrug',
 'total_mice',
 'tumor_mean',
 'tumor_median',
 'tumor_sem',
 'tumor_std',
 'tumor_variance']

## Bar and Pie Charts

In [82]:
# Generate a bar plot showing the total number of measurements taken on each drug regimen using pandas.



In [83]:
# Generate a bar plot showing the total number of measurements taken on each drug regimen using pyplot.



In [84]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [85]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [86]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [87]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [88]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [89]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [90]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


## Correlation and Regression

In [91]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen


### POTATO CODE: Keep for reference

In [92]:
# OPTION B
## KEEP - good code below but it doesn't fit the homework. It just drops part of the data for mouse g989 but keeps the mouse in the dataset.
## Keep for future reference if you want to pull out some data within a record but not remove the entire record.
#-------------------------
# Create a new dataframe that has the duplicate data removed from the original dataframe (all_data).
# Note: Code looks for duplicates in Mouse ID and Timepoint and put results in new dataframe (all_clean_data).

## DATAFRAME - all_data_clean_df
#all_data_clean_df = all_data_df.drop_duplicates(subset=["Mouse ID", "Timepoint"])
#print (all_data_clean_df)

# Note: Can also use this code below to confirm g989 is not in the cleaned data (it shows up in the original data set above)
# all_data_clean["Mouse ID"].value_counts()